In [3]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import timedelta
from openpyxl import Workbook

In [ ]:
# Mục tiêu: Nhóm các bản ghi theo số điện thoại và thời gian submit trong vòng 1 giờ
# Dữ liệu mẫu (thay thế bằng dữ liệu thực của bạn)
data = pd.read_excel('raw.xlsx', engine='openpyxl')
df = pd.DataFrame(data)

# Đảm bảo cột 'time' là kiểu datetime
# df['create_time'] = pd.to_datetime(df['create_time'])
# df['create_time'].fillna(pd.Timestamp('1970-01-01'), inplace=True)  # Thay thế giá trị NaN bằng thời gian mặc định
df['create_time'] = pd.to_datetime(df['create_time'])
# Sắp xếp theo số điện thoại và thời gian để tối ưu hóa việc tìm kiếm
df_sorted = df.sort_values(by=['ip_checkout', 'create_time']).reset_index(drop=True)

grouped_ids = []
processed_indices = set()

# Lặp qua từng số điện thoại duy nhất
for phone_num in df_sorted['ip_checkout'].unique():
    # Lấy tất cả các bản ghi cho số điện thoại này
    phone_records = df_sorted[df_sorted['ip_checkout'] == phone_num]

    # Lặp qua từng bản ghi trong nhóm số điện thoại
    for i in range(len(phone_records)):
        current_record = phone_records.iloc[i]
        current_id = current_record['buyer_id']
        current_time = current_record['create_time']

        # Nếu bản ghi này đã được xử lý trong một nhóm khác, bỏ qua
        if (phone_num, current_id) in processed_indices:
            continue

        # Bắt đầu một nhóm mới
        current_group_ids = [current_id]
        current_group_phone = current_record['ip_checkout']

        # Tìm các bản ghi khác trong cùng nhóm số điện thoại và trong vòng 1 giờ
        for j in range(i + 1, len(phone_records)):
            next_record = phone_records.iloc[j]
            next_id = next_record['buyer_id']
            next_time = next_record['create_time']

            # Nếu thời gian submit cách nhau trong vòng 1 giờ
            if abs(current_time - next_time) <= timedelta(hours=1):
                current_group_ids.append(next_id)
                # Đánh dấu bản ghi này là đã được xử lý
                processed_indices.add((current_group_phone, next_id))
            else:
                # Vì dữ liệu đã được sắp xếp, nếu khoảng thời gian vượt quá 1 giờ,
                # thì các bản ghi tiếp theo cũng sẽ vượt quá.
                break
        
        # Nếu nhóm có nhiều hơn 1 ID (tức là có ít nhất 2 ID cách nhau trong 1 giờ)
        if len(current_group_ids) > 3:
            grouped_ids.append({
                'phone_number': current_group_phone,
                'grouped_ids': sorted(list(set(current_group_ids))) # Loại bỏ trùng lặp và sắp xếp
            })
            # Đánh dấu ID gốc là đã được xử lý
            processed_indices.add((current_group_phone, current_id))

            
# In kết quả
# for group in grouped_ids:
#     print(f"Phone Number: {group['phone_number']}, Grouped IDs: {group['grouped_ids']}")

# saving output to an Excel file with source data and add a column for grouped IDs

output_wb = Workbook()
output_ws = output_wb.active
output_ws.append(['Phone Number', 'Grouped IDs'])
for group in grouped_ids:
    output_ws.append([group['phone_number'], ', '.join(map(str, group['grouped_ids']))])
# saving the original data with an additional column for grouped IDs
df['grouped_ids'] = df['buyer_id'].apply(lambda x: ', '.join(
    [str(group['grouped_ids']) for group in grouped_ids if x in group['grouped_ids']]
) if any(x in group['grouped_ids'] for group in grouped_ids) else '')
df.to_excel('original_data_with_grouped_ids.xlsx', index=False, engine='openpyxl')
# saving the original data with an additional column for grouped IDs

KeyError: 'creation_time'